In [1]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

path = 'data/'

In [2]:
# import the redcap data
# data\Deidentified REDCap Survey Data 2024.08.26.csv
redcap = pd.read_csv(path + 'Deidentified REDCap Survey Data 2024.08.26.csv')
redcap['visit_date'] = pd.to_datetime(redcap['visit_date']).dt.strftime('%Y-%m-%d')
temp = redcap.copy()
temp['visit_date'] = pd.to_datetime(temp['visit_date'])
temp = temp[['encrypt_mrn', 'ed_screened', 'visit_date', 'new_diagnosis', 'pcp_followup']]
temp = temp.drop_duplicates(subset='encrypt_mrn', keep='first')

In [3]:
# import the lab data
# data\Historical Lab Data File 2024.08.23.csv
lab = pd.read_csv(path + 'Historical Lab Data File 2024.08.23.csv')
lab['lab_date'] = pd.to_datetime(lab['lab_date']).dt.strftime('%Y-%m-%d')
lab = lab.rename(columns={'lab_date': 'visit_date'})

In [4]:
lab_group = lab.groupby(['encrypt_mrn', 'visit_date', 'lab_name']).agg(
    avg_value=('lab_value', 'mean'),
    max_value=('lab_value', 'max'),
    min_value=('lab_value', 'min')
).reset_index()

In [5]:
lab_table = lab_group.pivot_table(
    index=['encrypt_mrn', 'visit_date'], 
    columns='lab_name', 
    values=['avg_value', 'max_value', 'min_value']).reset_index()
lab_table.columns = [
    col[0] if col[0] in ['encrypt_mrn', 'visit_date'] else '_'.join(col).strip() 
    for col in lab_table.columns]

In [6]:
temp['visit_date'] = pd.to_datetime(temp['visit_date'], errors='coerce')
lab_table['visit_date'] = pd.to_datetime(lab_table['visit_date'], errors='coerce')
merged = pd.merge(temp, lab_table, on=['encrypt_mrn','visit_date'], how='left')
merged.head()

,encrypt_mrn,ed_screened,visit_date,new_diagnosis,pcp_followup,"avg_value_CHOLESTEROL, TOTAL",avg_value_CREATININE,avg_value_GLUCOSE,"avg_value_GLUCOSE, POC",avg_value_HDL CHOLESTEROL,...,max_value_HDL CHOLESTEROL,max_value_HEMOGLOBIN A1C,"max_value_LDL CHOLESTEROL, CALCULATED","min_value_CHOLESTEROL, TOTAL",min_value_CREATININE,min_value_GLUCOSE,"min_value_GLUCOSE, POC",min_value_HDL CHOLESTEROL,min_value_HEMOGLOBIN A1C,"min_value_LDL CHOLESTEROL, CALCULATED"
0,2303579,NYU Brooklyn,2023-01-10,Yes,Yes,268.0,0.74,305.0,269.333333,31.0,...,31.0,9.3,176.0,268.0,0.74,305.0,230.0,31.0,9.3,176.0
1,2114620,NYU Brooklyn,2022-01-11,No,No,NaN,0.70,386.0,324.000000,NaN,...,NaN,11.8,NaN,NaN,0.70,386.0,220.0,NaN,11.8,NaN
2,2116246,NYU Brooklyn,2022-01-12,No,Yes,NaN,0.74,401.0,300.666667,NaN,...,NaN,12.1,NaN,NaN,0.74,401.0,260.0,NaN,12.1,NaN
3,2183529,NYU Brooklyn,2023-01-12,No,Yes,NaN,0.64,214.0,228.000000,NaN,...,NaN,7.7,NaN,NaN,0.64,214.0,228.0,NaN,7.7,NaN
4,2117042,NYU Long Island,2022-01-13,Yes,Yes,NaN,NaN,NaN,443.000000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,443.0,NaN,NaN,NaN


In [7]:
# save as lab_data.csv
merged.to_csv('processed_data/lab_data.csv', index=False)